In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

DIVERSITY_UTILITY = float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = 0.5


In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
df

,Unnamed: 0,R,E_noise,E_minority,E_majority,M_noise,M_minority,M_majority,T_noise,T_minority,...,Y_unif,Y_reject_noise,Y,Y_raw,Y_reject,Y_reject_raw,D_p,key,stratum_utility,ml_outcomes
0,1,0,0.158618,0.158618,1.158618,1.214932,0.373550,1.373550,-1.736028,50,...,0.297061,-0.606825,1,0.797953,1,0.705484,0,0_60,0.60,0.60
1,33,0,0.929559,0.929559,1.929559,1.030761,0.960320,1.960320,-9.231331,49,...,0.758953,-0.912900,1,0.876568,1,0.811582,0,0_60,0.60,0.60
2,93,0,0.394233,0.394233,1.394233,-0.162916,-0.768683,0.231317,2.849825,51,...,0.057257,-1.373668,1,0.557573,1,0.433230,0,0_60,0.60,0.60
3,122,0,0.847899,0.847899,1.847899,0.232451,0.080350,1.080350,-3.511498,50,...,0.501282,-0.570888,1,0.746560,1,0.641148,0,0_60,0.60,0.60
4,179,0,-0.418852,-0.418852,0.581148,2.455702,1.036849,2.036849,-1.452177,51,...,0.502588,-0.393381,1,0.884612,1,0.823006,0,0_60,0.60,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,88718,0,3.072167,3.072167,4.072167,0.592295,2.664462,3.664462,10.416139,92,...,0.047716,-1.149934,1,0.975022,1,0.959475,0,0_106,0.98,0.98
99996,88564,0,3.785779,3.785779,4.785779,0.514379,3.300159,4.300159,10.936999,102,...,0.040181,-0.536345,1,0.986615,1,0.978122,0,0_118,0.98,0.98
99997,89639,1,3.230455,3.230455,4.230455,0.993001,3.223457,4.223457,8.873157,95,...,0.066753,-0.828693,1,0.961708,1,0.938397,1,1_95,1.19,1.19
99998,95298,1,-1.366077,-1.366077,-0.366077,-2.583951,-4.950028,-3.950028,-19.444103,12,...,0.781385,-0.635485,0,0.007033,0,0.004278,1,1_12,0.31,0.31


In [4]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [5]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [6]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [7]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [8]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,12,0.02,1
3,190,3,0,16,0.06,2
2,189,2,0,15,0.07,1
1,188,1,0,13,0.10,1
104,187,104,1,7,0.25,1
...,...,...,...,...,...,...
49,4,49,0,62,1074.56,1679
42,3,42,0,55,1091.00,2182
44,2,44,0,57,1107.54,2051
46,1,46,0,59,1115.34,1923


### Setup optimization problem 

In [9]:
from ortools.linear_solver import pywraplp


In [10]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [11]:
dff

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,12,0.02,1
1,188,1,0,13,0.10,1
2,189,2,0,15,0.07,1
3,190,3,0,16,0.06,2
4,183,4,0,17,0.48,8
...,...,...,...,...,...,...
187,172,187,1,95,1.19,1
188,166,188,1,97,1.25,1
189,167,189,1,98,1.25,1
190,169,190,1,101,1.23,1


In [12]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [13]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [14]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [15]:
# Now we have one constraint
solver.NumConstraints()

1

## Add CF Fair constraints

In [16]:
from collections import Counter

def convertListToProb(raw_list):
    counts = dict(Counter(raw_list))
    probs = {}
    for test_score in counts:
        probs[test_score]  = counts[test_score]/float(len(raw_list))
    return [(probs[t], t) for t in probs]

In [17]:
T_minoritys_list = df[df['R']==0][['T','T_minority_star']].groupby('T')['T_minority_star'].apply(list).reset_index(name='T_minoritys')


In [18]:
T_minoritys_list['probs'] = T_minoritys_list['T_minoritys'].apply(convertListToProb)


In [19]:
T_minoritys_list

,T,T_minoritys,probs
0,12,[12],"[(1.0, 12)]"
1,13,[14],"[(1.0, 14)]"
2,15,[14],"[(1.0, 14)]"
3,16,"[14, 13]","[(0.5, 14), (0.5, 13)]"
4,17,"[16, 18, 16, 16, 17, 16, 16, 16]","[(0.75, 16), (0.125, 18), (0.125, 17)]"
...,...,...,...
99,113,"[105, 105]","[(1.0, 105)]"
100,115,[105],"[(1.0, 105)]"
101,116,[108],"[(1.0, 108)]"
102,118,[110],"[(1.0, 110)]"


In [20]:
didntexist = 0
exists = 0
for ix, row in T_minoritys_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    majority_T = row['T']
    minoritys_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(0.0, majority_T)], -1.0)
    for prob in minoritys_Ts:
        if (1.0, prob[1]) not in vars_cache:
            vars_cache[(1.0, prob[1])] = solver.NumVar(0.0, 1.0, str((1.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
        cf_fair_stratum.SetCoefficient(vars_cache[(1.0, prob[1])], prob[0])
    

In [21]:
didntexist,exists

(9, 542)

In [22]:
T_majoritys_list = df[df['R']==1][['T','T_majority_star']].groupby('T')['T_majority_star'].apply(list).reset_index(name='T_majoritys')

In [23]:
T_majoritys_list['probs'] = T_majoritys_list['T_majoritys'].apply(convertListToProb)


In [24]:
didntexist = 0
exists = 0

for ix, row in T_majoritys_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    minority_T = row['T']
    majority_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(1.0, minority_T)], -1.0)
    for prob in majority_Ts:
        if (0.0, prob[1]) not in vars_cache:
            vars_cache[(0.0, prob[1])] = solver.NumVar(0.0, 1.0, str((0.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
            
        cf_fair_stratum.SetCoefficient(vars_cache[(0.0, prob[1])], prob[0])
    

In [25]:
cf_fair_stratum.basis_status

<bound method Constraint.basis_status of <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7feb50e962d0> >>

In [26]:
solver.NumConstraints()

193

## Solve linear program

In [27]:
solver.ABNORMAL

4

In [28]:
status = solver.Solve()


In [29]:
status

0

In [30]:
solver.OPTIMAL

0

In [31]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [32]:
df_decisions

,row_id,decision
0,0,0.000000
1,1,0.283597
2,2,0.283597
3,3,0.462698
4,4,0.522263
...,...,...
187,187,0.884360
188,188,0.961453
189,189,0.961453
190,190,1.000000


In [33]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
45,0,45,0,58,1123.36,2006,45,0.499629
46,1,46,0,59,1115.34,1923,46,0.499629
44,2,44,0,57,1107.54,2051,44,0.499629
42,3,42,0,55,1091.00,2182,42,0.499629
49,4,49,0,62,1074.56,1679,49,0.499629
...,...,...,...,...,...,...,...,...
104,187,104,1,7,0.25,1,104,0.000000
1,188,1,0,13,0.10,1,1,0.283597
2,189,2,0,15,0.07,1,2,0.283597
3,190,3,0,16,0.06,2,3,0.462698


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
45,0,45,0,58,1123.36,2006,45,0.499629
46,1,46,0,59,1115.34,1923,46,0.499629
44,2,44,0,57,1107.54,2051,44,0.499629
42,3,42,0,55,1091.00,2182,42,0.499629
49,4,49,0,62,1074.56,1679,49,0.499629
...,...,...,...,...,...,...,...,...
104,187,104,1,7,0.25,1,104,0.000000
1,188,1,0,13,0.10,1,1,0.283597
2,189,2,0,15,0.07,1,2,0.283597
3,190,3,0,16,0.06,2,3,0.462698


In [35]:
xxx.sort_values(by='decision',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
190,169,190,1,101,1.23,1,190,1.000000
100,177,100,0,115,1.00,1,100,1.000000
99,161,99,0,113,1.98,2,99,1.000000
191,168,191,1,112,1.25,1,191,1.000000
98,160,98,0,112,1.98,2,98,1.000000
...,...,...,...,...,...,...,...,...
1,188,1,0,13,0.10,1,1,0.283597
2,189,2,0,15,0.07,1,2,0.283597
105,186,105,1,12,0.31,1,105,0.000000
104,187,104,1,7,0.25,1,104,0.000000


In [36]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [37]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_minority_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()


In [38]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Path-Specific Fairness',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()


In [39]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.49999999999999983